# IMPORTS

In [ ]:
from keras.preprocessing import image
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras import layers, callbacks, utils, regularizers
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.utils import image_dataset_from_directory
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import os, pathlib, random
import numpy as np
import itertools

keras.utils.set_random_seed(42)

# CONSTANTS

In [ ]:
DATASET_DIR = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/Ro_Sign_language_Dataset")
MODEL_SAVE_DIR = pathlib.Path("/content/drive/MyDrive/Proiect Licenta/Saves/V2")
SAVE_NAME = f'Model with augmentation'

BATCH_SIZE = 64
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
VALIDATION_SPLIT = 0.2
EPOCHS = 50
UNITS = 256

# PREPROCESSING

In [ ]:
train_dataset = image_dataset_from_directory(DATASET_DIR / "train",
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             batch_size=BATCH_SIZE,
                                             label_mode='categorical')

validation_dataset = image_dataset_from_directory(DATASET_DIR / "validation",
                                                  image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                                  batch_size=BATCH_SIZE,
                                                  label_mode='categorical',
                                                  shuffle=False)

test_dataset = image_dataset_from_directory(DATASET_DIR / "test",
                                            image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                            batch_size=BATCH_SIZE,
                                            label_mode='categorical',
                                            shuffle=False)

In [ ]:
data_augmentation = keras.Sequential([layers.RandomFlip("horizontal"),
                                      layers.RandomRotation(0.05),
                                      layers.RandomZoom(0.2)])

rescale = layers.Rescaling(1./255)

# TRAINING

## ARCHITECTURE

In [ ]:
inputs = keras.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
x = data_augmentation(inputs)
x = rescale(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(UNITS, activation='relu')(x)
outputs = layers.Dense(26, activation="softmax")(x)
my_model = keras.Model(inputs=inputs, outputs=outputs)

my_model.summary()

In [ ]:
my_model.compile(loss="categorical_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath = str(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.keras'),
                                             save_best_only=True,
                                             monitor="val_loss")]

history = my_model.fit(train_dataset,
                       epochs=EPOCHS,
                       validation_data=validation_dataset,
                       callbacks=callbacks)

np.save(MODEL_SAVE_DIR / f'{SAVE_NAME}.history.npy', history.history)
my_model.save(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.tf', save_format='tf')
my_model.save(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.h5', save_format='h5')

In [ ]:
history = np.load(MODEL_SAVE_DIR / f'{SAVE_NAME}.history.npy', allow_pickle=True).item()

accuracy = history["accuracy"]
val_accuracy = history["val_accuracy"]
loss = history["loss"]
val_loss = history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Acuratețe antrenare")
plt.plot(epochs, val_accuracy, "b", label="Acuratețe validare")
plt.title("Acuratețe pe antrenare / validare")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Cost antrenare")
plt.plot(epochs, val_loss, "b", label="Cost validare")
plt.title("Cost pe antrenare / validare")
plt.legend()
plt.show()

## EVALUATION

In [ ]:
test_model = keras.models.load_model(MODEL_SAVE_DIR / f'{SAVE_NAME}.model.keras')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
class_names = test_dataset.class_names
y_pred = np.argmax(test_model.predict(test_dataset), axis=-1)
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_true = np.argmax(y_true, axis=-1)

print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plot_confusion_matrix(y_true, y_pred, classes=class_names)
plt.show()